In [ ]:
!pip install jupyter_contrib_nbextensions #instalando nb-extensions


In [2]:
#importando bibliotecas
import pandas as pd
import numpy as np


In [3]:
#importando csv pro collab
from google.colab import files
uploaded = files.upload()


Saving kc_house_data.csv to kc_house_data.csv


In [4]:
#lendo o arquivo csv e definindo uma váriavel pra ele
import io
data = pd.read_csv(io.BytesIO(uploaded['kc_house_data.csv']))


### EXEMPLOS DE ESTRUTURA DE CONTROLE

In [ ]:
#CONDICIONAIS 

        #### AND ####
# ex1 = data[(data['bedrooms'] == 2) & (data['floors'] == 2)]

        #### OR ####
# ex2 = data[(data['bedrooms'] == 2) | (data['floors'] == 5)]

        ## EXEMPLO DE UTILIZAÇÃO ##
data['niveis'] = 'NA'
data.loc[(data['price'] >= 0) & (data['price'] < 321950), 'niveis'] = 'nivel_0'
data.loc[(data['price'] >= 321950) & (data['price'] < 450000), 'niveis'] = 'nivel_1'
data.loc[(data['price'] >= 450000) & (data['price'] < 645000), 'niveis'] = 'nivel_2'
data.loc[(data['price'] >= 645000), 'niveis'] = 'nivel_3'

data[['price', 'niveis']].sample(15)


In [5]:
# LAÇO DE REPETIÇÃO - FOR  
# é necessário conhecer o tamanho do laço

for i in range(len(data)):
    if (data.loc[i, 'price'] >= 0) & (data.loc[i, 'price'] < 321950):
      data.loc[i, 'niveis'] = 'nivel_0'

    elif (data.loc[i, 'price'] >= 321950) & (data.loc[i, 'price'] < 450000):
      data.loc[i, 'niveis'] = 'nivel_1'

    elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
      data.loc[i, 'niveis'] = 'nivel_2'

    else:
      data.loc[i, 'niveis'] = 'nivel_3'


In [ ]:
# LAÇO DE REPETIÇÃO - WHILE

#mantem o fluxo de código continuamente, até que a condição seja satisfeita
#usado quando o tamanho do laço é desconhecido

x = 10
while x < 20:
    print(x)
    x += 1
else:
    print("fim while")


  RESPONDENDO AS PERGUNTAS DO CEO


In [ ]:
##Adicione as seguintes informações ao imóvel:
# - O nome da Rua
# - O número do Imóvel
# - O nome do Bairro
# - O nome da Cidade
# - O nome da Estado

# - Onde eu vou encontrar essas informações?  --- UTILIZAR API GEOPY
# - Qual dado em tenho na base que vai me ajudar a “linkar” essas novas informações. ---- LATITUDE E LONGITUDE
# - Coletar os novos dados.
# - Anexar os dados na base original.

from geopy.geocoders import Nominatim  # improtanto library 

geolocator = Nominatim(user_agent='geoapiExercises')

# response = geolocator.reverse('47.511,-122.257')  ## Funçao que retorna os dados do endereço de acordo com as coordenadas dadas

data['road'] = 'NA'
data['house_number'] = 'NA'
data['neighbourhood'] = 'NA'
data['city'] = 'NA'
data['state'] = 'NA'

for i in range(0, 10):
  coordinates = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
  
  response = geolocator.reverse(coordinates) #API request

  if 'house_number' in response.raw['address']:                                                   # o if é para caso a celula nao contenha a informaçao requerida, ele nao retornar nada e nao der erro.
        data.loc[i, 'house_number'] = response.raw['address']['house_number']          # se tiver ele popula a coluna, caso nao, mantem a celula vazia.
  if 'road' in response.raw['address']:  
        data.loc[i, 'road'] = response.raw['address']['road']
  if 'neighbourhood' in response.raw['address']:    
        data.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']
  if 'city' in response.raw['address']:    
        data.loc[i, 'city'] = response.raw['address']['city']
  if 'state' in response.raw['address']:    
        data.loc[i, 'state'] = response.raw['address']['state']

data.head()

In [ ]:
### COLOCANDO FILTROS NOS MAPAS

from matplotlib import pyplot as plt
import plotly.express as px

#### criação dos niveis
data['niveis'] = 'NA'

for i in range(len(data)):
    if (data.loc[i, 'price'] >= 0) & (data.loc[i, 'price'] < 321950):
      data.loc[i, 'niveis'] = 0

    elif (data.loc[i, 'price'] >= 321950) & (data.loc[i, 'price'] < 450000):
      data.loc[i, 'niveis'] = 1

    elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
      data.loc[i, 'niveis'] = 2

    else:
      data.loc[i, 'niveis'] = 3

data['niveis'] = data['niveis'].astype(int)

#### criando o mapa

house = data[['id', 'lat', 'long', 'price', 'niveis']].copy()

fig1 = px.scatter_mapbox(house,
                         lat='lat',
                         lon='long',
                         color='niveis',
                         size='price',
                         color_continuous_scale=px.colors.sequential.Inferno,
                         size_max=15,
                         zoom=10)

fig1.update_layout(mapbox_style = 'open-street-map')
fig1.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
fig1.show()

In [32]:
# CRIANDO FILTROS INTERATIVOS

import ipywidgets as widgets
from ipywidgets import fixed

data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else'no')

style = {'description_widht': 'initial'}

#botoes interativos

price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 7700000,                        ### botao de deslizar
    step = 1,
    description='Maximum Price',
    disable=False,
    style = style)

waterfront_bar = widgets.Dropdown(
    options=data['is_waterfront'].unique().tolist(),
    value='yes',
    disable=False
)

def update_map(data, waterfront, limit):
  houses = data[(data['price'] <= limit) & (data['is_waterfront'] == waterfront)][['id', 'lat', 'long', 'price', 'niveis']]

  fig1 = px.scatter_mapbox(houses,
                         lat='lat',
                         lon='long',
                         color='niveis',
                         size='price',
                         color_continuous_scale=px.colors.sequential.Inferno,
                         size_max=15,
                         zoom=10)

  fig1.update_layout(mapbox_style = 'open-street-map')
  fig1.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
  fig1.show()                                              

In [ ]:
### mostrando o mapa interativo

widgets.interactive(update_map, data=fixed(data), waterfront=waterfront_bar, limit=price_limit)

In [81]:
#### DASHBOARD INTERATIVO
from matplotlib import gridspec

#change data format
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

#widgets to control data
date_limit = widgets.SelectionSlider(
    options=data['date'].sort_values().unique().tolist(),
    value='2014-12-01',
    description = 'disponivel',
    continuous_update=False,
    orientation='horizontal',
    readout=True)

def update_map2(data, limit):                    ##definindo a funçao que torna os botoes interativos
  #filterdata
  data = data[data['date'] >= limit].copy()
  
  fig = plt.figure(figsize=(20,12))
  specs = gridspec.GridSpec (ncols=2, nrows=2, figure=fig)

  ax1 = fig.add_subplot(specs[0, :]) #First Row
  ax2 = fig.add_subplot(specs[1, 0]) #Second Row First Column
  ax3 = fig.add_subplot(specs[1, 1]) #Second row Second Column

  by_year = data[['id','year']].groupby('year').sum().reset_index()
  ax1.bar(by_year['year'], by_year['id'])

  by_day = data[['id','date']].groupby('date').mean().reset_index()
  ax2.plot(by_day['date'], by_day['id'])
  ax2.set_title('title: Avg price by day')
  plt.xticks(rotation=60)
  
  by_week = data[['id','year_week']].groupby('year_week').mean().reset_index()
  ax3.bar(by_week['year_week'], by_week['id'])
  ax3.set_title('title: Avg price by week')
  plt.xticks(rotation=60);

In [82]:
#vizualisando o dash interativo
widgets.interactive(update_map2, data=fixed(data), limit=date_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='disponivel', index=212, options=('…

EXERCÍCIOS

7. Exercícios:
Novas perguntas do CEO para você:

1. Qual a média do preço de compra dos
imóveis por “Nível”?
- Nível 0 -> Preço entre R$ 0 e R$321.950
- Nível 1 -> Preço entre R$ 321.950 eR$ 450.000
- Nível 2 -> Preço entre R$ 450.000 eR$ 645.000
- Nível 3 -> Acima de R$ 645.000

2. Qual a média do tamanho da sala deestar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e2550 sqft
- Size 3 -> Tamanho acima de 2550sqft

3. Adicione as seguinte informações ao
conjunto de dados original:
- Place ID: Identificação da localização
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala
de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do
porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção

5. Adicione os seguinte filtros no
Dashboard:
- Filtro por data disponível para
compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água
ou não



In [16]:
# 01. Qual a média do preço de compra dos imóveis por “Nível”?
# Nível 0 -> Preço entre R$0 e R$321.950
# Nível 1 -> Preço entre R$321.950 e R$450.000
# Nível 2 -> Preço entre R$450.000 e R$645.000
# Nível 3 -> Acima de R$ 645.000

data[['price', 'niveis']].groupby('niveis').mean().reset_index()


# niveis	price
# 0	nivel_0	251544.621507
# 1	nivel_1	383585.205332
# 2	nivel_2	536164.422840
# 3	nivel_3	985008.983189


,niveis,price
0,nivel_0,251544.621507
1,nivel_1,383585.205332
2,nivel_2,536164.422840
3,nivel_3,985008.983189


In [ ]:
# Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
# Size 0 -> Tamanho entre 0 e 1427sqft
# Size 1 -> Tamanho entre 1427 e 1910 sqft
# Size 2 -> Tamanho entre 1910 e2550 sqft
# Size 3 -> Tamanho acima de 2550sqft

data['size_level'] = 'NA'

for i in range(len(data)):
    if (data.loc[i, 'sqft_living'] >= 0) & (data.loc[i, 'sqft_living'] < 1427):
      data.loc[i, 'size_level'] = 'level_0'

    elif (data.loc[i, 'sqft_living'] >= 1427) & (data.loc[i, 'sqft_living'] < 1910):
      data.loc[i, 'size_level'] = 'level_1'

    elif (data.loc[i, 'price'] >= 1910) & (data.loc[i, 'sqft_living'] < 2550):
      data.loc[i, 'size_level'] = 'level_2'
    else:
      data.loc[i, 'size_level'] = 'level_3'

data[['sqft_living', 'size_level']].groupby('size_level').mean().reset_index()

# saida
# size_level  sqft_living
# 	level_0	  1123.776421
# 	level_1 	1661.198088
# 	level_2	  2202.700239
# 	level_3 	3318.988404

In [ ]:
# 3. Adicione as seguinte informações ao conjunto de dados original:
# Place ID: Identificação da localização       'place_id'
# OSM Type: Open Street Map type               'osm_type'
# Country: Nome do País                        'country_code'
# Country Code: Código do País                 'country'

from geopy.geocoders import Nominatim  # improtanto library 

geolocator = Nominatim(user_agent='geoapiExercises')

data['place_id'] = 'NA'
data['osm_type'] = 'NA'
data['country_code'] = 'NA'
data['country'] = 'NA'


for i in range(0, 10):
  coordinates = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
  
  response = geolocator.reverse(coordinates) #API request

  if 'place_id' in response.raw:                                                   # o if é para caso a celula nao contenha a informaçao requerida, ele nao retornar nada e nao der erro.
        data.loc[i, 'place_id'] = response.raw['place_id']          # se tiver ele popula a coluna, caso nao, mantem a celula vazia.
  if 'osm_type' in response.raw:  
        data.loc[i, 'osm_type'] = response.raw['osm_type']
  if 'country_code' in response.raw['address']:    
        data.loc[i, 'country_code'] = response.raw['address']['country_code']
  if 'city' in response.raw['address']:    
        data.loc[i, 'country'] = response.raw['address']['country']

data.head()


In [78]:
# 4. Adicione os seguinte filtros no Mapa:
# -Tamanho mínimo da área da sala de estar.  ---sqft_living_min 
# -Número mínimo de banheiros.               ---bathrooms_min
# -Valor Máximo do Preço.                    ---price_limit
# -Tamanho máximo da área do porão.          ---sqft_basement_max
# -Filtro das Condições do Imóvel.           ---condition_status_bar
# -Filtro por Ano de Construção              ---year_built_bar

import ipywidgets as widgets
from ipywidgets import fixed

style = {'description_widht': 'initial'}

data['condition_status'] = data['condition'].sort_values(ascending=True).apply( lambda x: 'critical' if x == 1 else
                                                              'poor' if x == 2 else
                                                              'regular' if x == 3 else
                                                              'good' if x == 4 else
                                                              'excellent' if x == 5 else 'NA')

#botoes interativos
sqft_living_min = widgets.IntSlider(
    value = 2000,
    min = 290,
    max = 13540,                        ### botao de deslizar
    step = 1,
    description='Minimum living room size',
    disable=False,
    style = style)

bathrooms_min = widgets.IntSlider(
    value = 4,
    min = 0,
    max = 8,                        ### botao de deslizar
    step = 1,
    description='Minimum number of bathrooms',
    disable=False,
    style = style)

price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 7700000,                        ### botao de deslizar
    step = 1,
    description='Maximum Price',
    disable=False,
    style = style)

sqft_basement_max = widgets.IntSlider(
    value = 0,
    min = 0,
    max = 4820,                        ### botao de deslizar
    step = 1,
    description='Maximum basement size',
    disable=False,
    style = style)

condition_status_bar = widgets.Dropdown(
    options=data['condition_status'].unique().tolist(),
    value='regular',
    disable=False)

year_built_bar = widgets.Dropdown(
    options=data['yr_built'].sort_values(ascending=True).unique().tolist(),
    value=1930,
    disable=False)

def update_map3(data, living_min, bath_min, basement_max, status, built, limit):
  houses = data[(data['sqft_living'] >= living_min ) &
                (data['bathrooms'] >= bath_min ) &
                (data['sqft_basement'] <= basement_max) &
                (data['price'] <= limit) & 
                (data['condition_status'] == status ) &
                (data['yr_built'] == built)][['id', 'lat', 'long', 'price', 'niveis']]

  fig1 = px.scatter_mapbox(houses,
                         lat='lat',
                         lon='long',
                         color='niveis',
                         size='price',
                         color_continuous_scale=px.colors.sequential.Inferno,
                         size_max=15,
                         zoom=10)

  fig1.update_layout(mapbox_style = 'open-street-map')
  fig1.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
  fig1.show()                       

In [ ]:
### mostrando os filtro do mapa
widgets.interactive(update_map3, data=fixed(data),
                    living_min=sqft_living_min, 
                    bath_min=bathrooms_min, 
                    basement_max=sqft_basement_max, 
                    status=condition_status_bar, 
                    built=year_built_bar,
                    limit=price_limit)

In [83]:
data['yr_renovated'].unique()

array([   0, 1991, 2002, 2010, 1999, 1992, 2013, 1994, 1978, 2005, 2008,
       2003, 1984, 1954, 2014, 2011, 1974, 1983, 1945, 1990, 1988, 1957,
       1977, 1981, 1995, 2000, 1998, 1970, 1989, 2004, 1986, 2009, 2007,
       1987, 1973, 2006, 1985, 2001, 1980, 1971, 1979, 1997, 1950, 1969,
       1948, 2015, 1968, 2012, 1963, 1951, 1993, 1962, 1996, 1972, 1953,
       1955, 1982, 1956, 1940, 1976, 1946, 1975, 1958, 1964, 1959, 1960,
       1967, 1965, 1934, 1944])

In [85]:
# 5.Adicione os seguinte filtros no Dashboard:
# Filtro por data disponível para compra.
# Filtro por ano de renovação.
# Filtro se possui vista para a água ou não

from matplotlib import gridspec

#change data format
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

#widgets to control data
date_limit = widgets.SelectionSlider(
    options=data['date'].sort_values().unique().tolist(),
    value='2014-12-01',
    description = 'DISPONIBLE DATE',
    continuous_update=False,
    orientation='horizontal',
    readout=True)

date_renovated_min = widgets.SelectionSlider(
    options=data['yr_renovated'].sort_values().unique().tolist(),
    value=2000,
    description = 'LAST RENOVATION',
    continuous_update=False,
    orientation='horizontal',
    readout=True)

waterfront_bar = widgets.Dropdown(
    options=data['is_waterfront'].unique().tolist(),
    value='yes',
    description = 'WATERFRONT',
    disable=False)

def update_map4(data, limit, renovated, waterfront):                    ##definindo a funçao que torna os botoes interativos
  #filterdata
  data = data[(data['date'] >= limit) &
              (data['yr_renovated'] >= renovated) &
              (data['is_waterfront'] == waterfront)].copy()
  
  fig = plt.figure(figsize=(20,12))
  specs = gridspec.GridSpec (ncols=2, nrows=2, figure=fig)

#### DASHBOARD
  ax1 = fig.add_subplot(specs[0, :]) #First Row
  ax2 = fig.add_subplot(specs[1, 0]) #Second Row First Column
  ax3 = fig.add_subplot(specs[1, 1]) #Second row Second Column

  by_year = data[['id','year']].groupby('year').sum().reset_index()   #GRAFICO 1
  ax1.bar(by_year['year'], by_year['id'])

  by_day = data[['id','date']].groupby('date').mean().reset_index()   #GRAFICO 2
  ax2.plot(by_day['date'], by_day['id'])
  ax2.set_title('title: Avg price by day')
  plt.xticks(rotation=60)
  
  by_week = data[['id','year_week']].groupby('year_week').mean().reset_index()  #GRAFICO 3
  ax3.bar(by_week['year_week'], by_week['id'])
  ax3.set_title('title: Avg price by week')
  plt.xticks(rotation=60);


In [86]:
#mostrando dashboard com filtros interativos
widgets.interactive(update_map4, data=fixed(data), limit=date_limit, renovated=date_renovated_min, waterfront=waterfront_bar)

interactive(children=(SelectionSlider(continuous_update=False, description='DISPONIBLE DATE', index=212, optio…